In [2]:
!git clone https://github.com/bardiarms/gan-rl.git
%cd gan-rl

Cloning into 'gan-rl'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 60 (delta 29), reused 46 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (60/60), 312.80 KiB | 1.85 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/gan-rl


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install torch torchvision torchaudio tqdm matplotlib pandas pillow

In [4]:
DATA_DIR = "/content/drive/MyDrive/gan-rl-data"
RUN_DIR  = "/content/drive/MyDrive/gan-rl-runs"

In [5]:
!ls -la "$DATA_DIR"

ls: /content/drive/MyDrive/gan-rl-data/cartoonset100k: No such file or directory
total 0
lrw------- 1 root root 0 Dec 23 16:41 cartoonset100k -> /content/drive/.shortcut-targets-by-id/1l-9xzrKNZUKfWTFGh0SRZe9j1HKLRRqJ/cartoonset100k


In [6]:
REPO_DIR = "/content/gan-rl"

In [7]:
import os

In [8]:
os.makedirs(RUN_DIR, exist_ok=True)
print("DATA_DIR exists:", os.path.exists(DATA_DIR))
print("RUN_DIR:", RUN_DIR)

DATA_DIR exists: True
RUN_DIR: /content/drive/MyDrive/gan-rl-runs


In [9]:
# DATA_ROOT = "/content/cartoonset100k"
DATA_ROOT = "/content/drive/MyDrive/gan-rl-data/cartoonset100k"

In [10]:
from pathlib import Path


In [11]:
data_root = Path(DATA_ROOT)

pairs, missing_meta, missing_img = [], [], []

for d in sorted(data_root.iterdir()):

    for png_path in d.glob("*.png"):
        csv_path = png_path.with_suffix(".csv")
        if csv_path.exists():
            pairs.append((str(png_path), str(csv_path), int(d.name)))   # If the pair exists, add them to pairs
        else:
            missing_meta.append(str(png_path))


print("Total pairs:", len(pairs))
print("Missing CSV for PNG:", len(missing_meta))

Total pairs: 100000
Missing CSV for PNG: 17


In [12]:
for i in range(5):
    print(pairs[i])

('/content/drive/MyDrive/gan-rl-data/cartoonset100k/0/cs12694632614924764393.png', '/content/drive/MyDrive/gan-rl-data/cartoonset100k/0/cs12694632614924764393.csv', 0)
('/content/drive/MyDrive/gan-rl-data/cartoonset100k/0/cs12110888894984629978.png', '/content/drive/MyDrive/gan-rl-data/cartoonset100k/0/cs12110888894984629978.csv', 0)
('/content/drive/MyDrive/gan-rl-data/cartoonset100k/0/cs12783151780612292884.png', '/content/drive/MyDrive/gan-rl-data/cartoonset100k/0/cs12783151780612292884.csv', 0)
('/content/drive/MyDrive/gan-rl-data/cartoonset100k/0/cs1182038763634098519.png', '/content/drive/MyDrive/gan-rl-data/cartoonset100k/0/cs1182038763634098519.csv', 0)
('/content/drive/MyDrive/gan-rl-data/cartoonset100k/0/cs1213446833171185255.png', '/content/drive/MyDrive/gan-rl-data/cartoonset100k/0/cs1213446833171185255.csv', 0)


In [13]:
pairs.sort(key=lambda x: x[0])  # sort by image path

In [14]:
# Assert we can open the first image + read first metadata line
from PIL import Image
import pandas as pd

img_path, meta_path, folder_id = pairs[0]
img = Image.open(img_path).convert("RGB")
df = pd.read_csv(meta_path, header=None, names=["attr", "value", "max"])

print("Sample folder:", folder_id)
print("Image size:", img.size)
print("Metadata shape:", df.shape)
print(df.head())

Sample folder: 0
Image size: (500, 500)
Metadata shape: (18, 3)
             attr  value  max
0       eye_angle      2    3
1      eye_lashes      1    2
2         eye_lid      1    2
3     chin_length      0    3
4  eyebrow_weight      0    2


In [15]:
# Read one metadata file
def read_meta_csv(meta_path: str) -> pd.DataFrame:
    df = pd.read_csv(meta_path, header=None, names=["attr", "value", "max"])
    # ensure types
    df["attr"] = df["attr"].astype(str)
    df["value"] = df["value"].astype(int)
    df["max"] = df["max"].astype(int)
    return df

In [16]:
# Buil a schema for one-hot encoding
def build_schema(pairs, max_files=20):
    # pairs: list of (img_path, meta_path, folder_id)
    attr_to_num_classes = {}  # attr -> (max+1)
    attr_order = []           # stable order of attrs as discovered

    for i, (_, meta_path, _) in enumerate(pairs[:max_files]):
        df = read_meta_csv(meta_path)
        for _, row in df.iterrows():
            attr = row["attr"]
            num_classes = row["max"] + 1

            if attr not in attr_to_num_classes:
                attr_to_num_classes[attr] = num_classes
                attr_order.append(attr)
            else:
                # keep the maximum seen (in case some files differ)
                attr_to_num_classes[attr] = max(attr_to_num_classes[attr], num_classes)

    # offsets for packing one big vector
    offsets = {}
    total_dim = 0
    for attr in attr_order:
        offsets[attr] = total_dim
        total_dim += attr_to_num_classes[attr]

    return attr_order, attr_to_num_classes, offsets, total_dim

In [17]:
attr_order, attr_to_num_classes, offsets, meta_dim = build_schema(pairs)
print("num attributes:", len(attr_order))
print("meta vector dim:", meta_dim)
print(attr_order[:])

num attributes: 18
meta vector dim: 235
['eye_angle', 'eye_lashes', 'eye_lid', 'chin_length', 'eyebrow_weight', 'eyebrow_shape', 'eyebrow_thickness', 'face_shape', 'facial_hair', 'hair', 'eye_color', 'face_color', 'hair_color', 'glasses', 'glasses_color', 'eye_slant', 'eyebrow_width', 'eye_eyebrow_distance']


In [18]:
# Freeze the schema (Run Once)

import json

schema = {
    "attr_order": attr_order,
    "attr_to_num_classes": attr_to_num_classes,
    "offsets": offsets,
    "total_dim": meta_dim,
}

SCHEMA_PATH = "/content/drive/MyDrive/gan-rl-runs/meta_schema.json"
os.makedirs(os.path.dirname(SCHEMA_PATH), exist_ok=True)

with open(SCHEMA_PATH, "w") as f:
    json.dump(schema, f, indent=2)

print("Saved:", SCHEMA_PATH)

Saved: /content/drive/MyDrive/gan-rl-runs/meta_schema.json


In [19]:
from torchvision import transforms

In [20]:

IMG_SIZE = 64      # We convert 500*500 pixel images into 128*128.
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

img_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
    ])

In [21]:
import torch
import numpy as np

In [22]:
# Read metadata and store them in pandas dataframe
def read_meta(meta_path: str)-> pd.DataFrame:

  df = pd.read_csv(meta_path, header=None, names=["attr", "value", "max"])
  df["attr"] = df["attr"].astype(str)
  df["value"] = df["value"].astype(int)
  df["max"] = df["max"].astype(int)

  return df

In [23]:
# Apply one-hot encoding
def encode_onehot(meta_path: str,
                  attr_to_num_classes: dict,
                  offsets: dict,
                  total_dim: int
                  )-> torch.Tensor:

    df = read_meta(meta_path)
    vec = np.zeros((total_dim,), dtype=np.float32)

    for _, row in df.iterrows():
        attr = row["attr"]
        val = int(row["value"])

        if attr not in offsets:
          continue

        n = attr_to_num_classes[attr]
        if val < 0 or val >= n:
            val = max(0, min(val, n - 1))

        vec[offsets[attr] + val] = 1.0

    return torch.from_numpy(vec)


In [24]:
from torch.utils.data import Dataset

In [25]:
class CartoonSetDataset(Dataset):

    def __init__(self, pairs, img_transform, meta_cache = None):
        self.pairs = pairs
        self.img_transform = img_transform
        self.meta_cache = meta_cache

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img_path, meta_path, folder_id = self.pairs[idx]

        img = Image.open(img_path).convert("RGB")
        img = self.img_transform(img)

        if self.meta_cache is not None:
            meta = self.meta_cache[meta_path]
            return img, meta, folder_id

        else:
            return img, folder_id



In [26]:
from torch.utils.data import DataLoader

In [41]:
ds = CartoonSetDataset(pairs=pairs, img_transform=img_transform)

dl = DataLoader(
    ds,
    batch_size=4,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=False
)
imgs, folder_ids = next(iter(dl))
print("imgs:", imgs.shape, imgs.min().item(), imgs.max().item())
#print("metas:", metas.shape, metas.min().item(), metas.max().item())
print("folder_ids:", folder_ids[:8])
#print("meta sums (first 8):", metas[:8].sum(dim=1))

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


imgs: torch.Size([4, 3, 64, 64]) -1.0 1.0
folder_ids: tensor([2, 2, 0, 7])


In [28]:
import torch
import torch.nn as nn
from src import models
IMG_SIZE = 64
Z_DIM = 128
BATCH_SIZE = 4
DEVICE = "cuda"
criterion = nn.BCEWithLogitsLoss()

In [29]:
G = models.Generator_64(z_dim=Z_DIM).to(DEVICE)
D = models.Discriminator_64().to(DEVICE)

imgs, folder_ids = next(iter(dl))  # from your existing DataLoader
imgs = imgs.to(DEVICE)

z = torch.randn(imgs.size(0), Z_DIM, device=DEVICE)
fake = G(z)

print("Real:", imgs.shape, imgs.min().item(), imgs.max().item())
print("Fake:", fake.shape, fake.min().item(), fake.max().item())

d_real = D(imgs)
d_fake = D(fake.detach())

print("D(real) shape:", d_real.shape, "min/max:", d_real.min().item(), d_real.max().item())
print("D(fake) shape:", d_fake.shape, "min/max:", d_fake.min().item(), d_fake.max().item())

Real: torch.Size([4, 3, 64, 64]) -0.9607843160629272 1.0
Fake: torch.Size([4, 3, 64, 64]) -0.9999998807907104 0.9999997019767761
D(real) shape: torch.Size([4]) min/max: -0.09946166723966599 -0.0696922019124031
D(fake) shape: torch.Size([4]) min/max: -0.13368116319179535 -0.022538205608725548


In [30]:
lr_G = 2e-4
lr_D = 1e-4
beta1, beta2 = 0.5, 0.999

opt_D = torch.optim.Adam(D.parameters(), lr=lr_D, betas=(beta1, beta2))
opt_G = torch.optim.Adam(G.parameters(), lr=lr_G, betas=(beta1, beta2))

use_amp = True
scaler_D = torch.amp.GradScaler(device="cuda", enabled=use_amp)
scaler_G = torch.amp.GradScaler(device="cuda", enabled=use_amp)

### Train GAN

In [31]:
from torchvision.utils import make_grid, save_image

In [32]:
# Denormalize pixels for viewing
def denorm(x):
    return (x * 0.5 + 0.5).clamp(0, 1)

@torch.no_grad()
def save_samples(G, step, fixed_z, out_dir, nrow=8):
    G.eval()
    fake = G(fixed_z)
    grid = make_grid(denorm(fake), nrow=nrow)
    path = os.path.join(out_dir, f"step_{step:06d}.png")
    save_image(grid, path)
    G.train()



In [33]:
# Add Gaussian Noise to Discriminator's inputs
def noise_sigma(step, sigma0=0.10, hold_steps=1500, decay_steps=4000):
    if step < hold_steps:
        return sigma0
    t = (step - hold_steps) / decay_steps
    return sigma0 * max(0.0, 1.0 - t)

# Noise Helper
def add_instance_noise(x, sigma):
    if sigma <= 0:
        return x
    return x + sigma * torch.randn_like(x)

In [34]:
def train_func(RUN_DIR: str,
               iters: int,
               SAMPLE_EVERY: int,
               CHKPT_EVERY: int
               )-> None:

    SAMPLES_DIR = os.path.join(RUN_DIR, f"samples_128_{iters}_iters")
    CHKPT_DIR = os.path.join(RUN_DIR, f"checkpoints_128_{iters}_iters")
    os.makedirs(SAMPLES_DIR, exist_ok=True)
    os.makedirs(CHKPT_DIR, exist_ok=True)

    fixed_z = torch.randn(64, Z_DIM, device=DEVICE)

    G.train(); D.train()

    step = 0
    data_iter = iter(dl)

    while step < iters:
        try:
            imgs, folder_ids = next(data_iter)
        except StopIteration:
            data_iter = iter(dl)
            imgs, folder_ids = next(data_iter)

        real = imgs.to(DEVICE, non_blocking=True)
        B = real.size(0)

        real_labels = torch.ones(B, device=DEVICE)
        fake_labels = torch.zeros(B, device=DEVICE)


        # ---Train Discriminator---

        opt_D.zero_grad(set_to_none=True)
        z = torch.randn(B, Z_DIM, device=DEVICE)

        sigma = 0 #noise_sigma(step)


        with torch.amp.autocast(device_type="cuda", enabled=use_amp):
            fake = G(z)

            real_in = add_instance_noise(real, sigma)
            fake_in = add_instance_noise(fake.detach(), sigma)

            logits_real = D(real_in)
            logits_fake = D(fake_in)
            loss_D_real = criterion(logits_real, real_labels)
            loss_D_fake = criterion(logits_fake, fake_labels)
            loss_D = loss_D_real + loss_D_fake

        scaler_D.scale(loss_D).backward()
        scaler_D.step(opt_D)
        scaler_D.update()


        # -------------------------
        # Train Generator
        # -------------------------
        # for _ in range(2):          # Generator updates twice as Discriminator
        opt_G.zero_grad(set_to_none=True)
        z = torch.randn(B, Z_DIM, device=DEVICE)

        with torch.amp.autocast(device_type="cuda", enabled=use_amp):
            fake = G(z)
            #fake_in = add_instance_noise(fake, sigma)
            logits_fake_for_G = D(fake)
            loss_G = criterion(logits_fake_for_G, real_labels)

        scaler_G.scale(loss_G).backward()
        scaler_G.step(opt_G)
        scaler_G.update()

        # Logging
        if step % 10 == 0:
            print(
                f"step {step:04d} | "
                f"loss_D {loss_D.item():.4f} (r {loss_D_real.item():.4f}, f {loss_D_fake.item():.4f}) | "
                f"loss_G {loss_G.item():.4f} | "
                f"D(real) {logits_real.mean().item():+.3f} | D(fake) {logits_fake.mean().item():+.3f}"
            )

        # Save samples
        if step % SAMPLE_EVERY == 0:
            save_samples(G, step, fixed_z, out_dir = SAMPLES_DIR)

        # Save checkpoints (optional)
        if step > 0 and step % CHKPT_EVERY == 0:
            ckpt_path = os.path.join(CHKPT_DIR, f"gan_step_{step:06d}.pt")
            torch.save(
                {
                    "step": step,
                    "G": G.state_dict(),
                    "D": D.state_dict(),
                    "opt_G": opt_G.state_dict(),
                    "opt_D": opt_D.state_dict(),
                    "scaler_D": scaler_D.state_dict(),
                    "scaler_G": scaler_G.state_dict(),
                },
                ckpt_path,
            )

        step += 1


In [1]:
train_func(RUN_DIR = "/content/drive/MyDrive/gan-rl-runs/12000_iters", iters = 12000, SAMPLE_EVERY = 500, CHKPT_EVERY = 500)

NameError: name 'train_func' is not defined

## Resume Train From Checkpoint

In [42]:

def resume_from_ckpt(
    ckpt_path: str,
    total_iters: int,
    RUN_DIR: str,
    SAMPLE_EVERY: int,
    CHKPT_EVERY: int,
    dl,
    G,
    D,
    opt_G,
    opt_D,
    criterion,
    save_samples,
    DEVICE="cuda",
    Z_DIM=128,
    use_amp=True,
    scaler_D=None,
    scaler_G=None
):
    # ---- load ----
    ckpt = torch.load(ckpt_path, map_location=DEVICE)
    G.load_state_dict(ckpt["G"])
    D.load_state_dict(ckpt["D"])
    opt_G.load_state_dict(ckpt["opt_G"])
    opt_D.load_state_dict(ckpt["opt_D"])

    start_step = int(ckpt.get("step", 0)) + 1

    # if scaler is None:
    #     scaler = torch.amp.GradScaler(device_type="cuda", enabled=use_amp)
    if "scaler_D" in ckpt and ckpt["scaler_D"] is not None:
        try:
            scaler_D.load_state_dict(ckpt["scaler_D"])
        except Exception:
            pass

    if "scaler_G" in ckpt and ckpt["scaler_G"] is not None:
        try:
            scaler_G.load_state_dict(ckpt["scaler_G"])
        except Exception:
            pass

    # ---- dirs ----
    SAMPLES_DIR = os.path.join(RUN_DIR, f"samples_64_{total_iters}_iters")
    CKPT_DIR    = os.path.join(RUN_DIR, f"checkpoints_64_{total_iters}_iters")
    os.makedirs(SAMPLES_DIR, exist_ok=True)
    os.makedirs(CKPT_DIR, exist_ok=True)

    print(f"✅ Loaded: {ckpt_path}")
    print(f"▶️ Resume: step {start_step} -> {total_iters-1}")

    fixed_z = torch.randn(64, Z_DIM, device=DEVICE)

    G.train(); D.train()
    step = start_step
    data_iter = iter(dl)

    while step < total_iters:
        try:
            imgs, folder_ids = next(data_iter)
        except StopIteration:
            data_iter = iter(dl)
            imgs, folder_ids = next(data_iter)

        real = imgs.to(DEVICE, non_blocking=True)
        B = real.size(0)

        real_labels = torch.ones(B, device=DEVICE)
        fake_labels = torch.zeros(B, device=DEVICE)


        # ---Train Discriminator---

        opt_D.zero_grad(set_to_none=True)
        z = torch.randn(B, Z_DIM, device=DEVICE)

        sigma = 0 #noise_sigma(step)


        with torch.amp.autocast(device_type="cuda", enabled=use_amp):
            fake = G(z)

            real_in = add_instance_noise(real, sigma)
            fake_in = add_instance_noise(fake.detach(), sigma)

            logits_real = D(real_in)
            logits_fake = D(fake_in)
            loss_D_real = criterion(logits_real, real_labels)
            loss_D_fake = criterion(logits_fake, fake_labels)
            loss_D = loss_D_real + loss_D_fake

        scaler_D.scale(loss_D).backward()
        scaler_D.step(opt_D)
        scaler_D.update()


        # -------------------------
        # Train Generator
        # -------------------------
        # for _ in range(2):          # Generator updates twice as Discriminator
        opt_G.zero_grad(set_to_none=True)
        z = torch.randn(B, Z_DIM, device=DEVICE)

        with torch.amp.autocast(device_type="cuda", enabled=use_amp):
            fake = G(z)
            #fake_in = add_instance_noise(fake, sigma)
            logits_fake_for_G = D(fake)
            loss_G = criterion(logits_fake_for_G, real_labels)

        scaler_G.scale(loss_G).backward()
        scaler_G.step(opt_G)
        scaler_G.update()

        if step % 10 == 0:
            print(
                f"step {step:04d} | loss_D {loss_D.item():.4f} | loss_G {loss_G.item():.4f} | "
                f"D(real) {logits_real.mean().item():+.3f} | D(fake) {logits_fake.mean().item():+.3f}"
            )

        if step % SAMPLE_EVERY == 0:
            save_samples(G, step, fixed_z, out_dir=SAMPLES_DIR)

        if step > 0 and step % CHKPT_EVERY == 0:
            out_path = os.path.join(CKPT_DIR, f"gan_step_{step:06d}.pt")
            torch.save(
                {
                    "step": step,
                    "G": G.state_dict(),
                    "D": D.state_dict(),
                    "opt_G": opt_G.state_dict(),
                    "opt_D": opt_D.state_dict(),
                    "scaler_D": scaler_D.state_dict(),
                    "scaler_G": scaler_G.state_dict(),
                },
                out_path,
            )
            print("💾 Saved:", out_path)

        step += 1

In [43]:
resume_from_ckpt(
    ckpt_path="/content/drive/MyDrive/gan-rl-runs/12000_iters/checkpoints_128_12000_iters/gan_step_002500.pt",
    total_iters=12000,
    RUN_DIR="/content/drive/MyDrive/gan-rl-runs/12000_iters/checkpoints_64_12000_iters",
    SAMPLE_EVERY=500,
    CHKPT_EVERY=500,
    dl=dl, G=G, D=D,
    opt_G=opt_G, opt_D=opt_D,
    criterion=criterion,
    save_samples=save_samples,
    DEVICE=DEVICE,
    Z_DIM=Z_DIM,
    use_amp=use_amp,
    scaler_D=scaler_D,
    scaler_G=scaler_G
)

✅ Loaded: /content/drive/MyDrive/gan-rl-runs/12000_iters/checkpoints_128_12000_iters/gan_step_002500.pt
▶️ Resume: step 2501 -> 11999
step 2510 | loss_D 1.2668 | loss_G 0.7779 | D(real) +0.051 | D(fake) -0.241
step 2520 | loss_D 0.8847 | loss_G 1.1308 | D(real) +1.030 | D(fake) -0.321
step 2530 | loss_D 0.8540 | loss_G 1.2237 | D(real) +0.876 | D(fake) -0.459
step 2540 | loss_D 1.2627 | loss_G 1.0067 | D(real) -0.065 | D(fake) -0.349
step 2550 | loss_D 1.3379 | loss_G 0.7821 | D(real) +0.273 | D(fake) +0.090
step 2560 | loss_D 1.4662 | loss_G 0.8350 | D(real) -0.111 | D(fake) +0.026
step 2570 | loss_D 1.4360 | loss_G 0.9077 | D(real) -0.210 | D(fake) -0.173
step 2580 | loss_D 1.2204 | loss_G 1.0562 | D(real) +0.464 | D(fake) +0.014
step 2590 | loss_D 1.4519 | loss_G 0.9529 | D(real) -0.163 | D(fake) -0.081
step 2600 | loss_D 1.0611 | loss_G 0.9488 | D(real) +0.577 | D(fake) -0.197
step 2610 | loss_D 1.4531 | loss_G 0.8496 | D(real) -0.121 | D(fake) -0.003
step 2620 | loss_D 1.5839 | lo

# Diagnosis

In [ ]:
from torchvision.utils import make_grid, save_image

def denorm(x):
    # [-1, 1] -> [0, 1]
    return (x * 0.5 + 0.5).clamp(0, 1)

@torch.no_grad()
def save_random_grid(G, step, out_dir, z_dim, n=64, nrow=8, device="cuda"):
    G.eval()
    os.makedirs(out_dir, exist_ok=True)

    z = torch.randn(n, z_dim, device=device)      # fresh random z
    fake = G(z)                                   # [n, 3, H, W] in [-1,1]
    grid = make_grid(denorm(fake), nrow=nrow)

    path = os.path.join(out_dir, f"random_step_{step:06d}.png")
    save_image(grid, path)

    G.train()
    print("Saved:", path)


In [ ]:
SAMPLES_DIR = "/content/drive/MyDrive/gan-rl-runs/12000_iters/samples_128_12000_iters"

In [ ]:
save_random_grid(G, step=500, out_dir=SAMPLES_DIR, z_dim=Z_DIM, device=DEVICE)